In [23]:
import pandas as pd
from tqdm import tqdm
from os.path import join as oj
data_dir = '../data/hrsa/data_AHRF_2018-2019/'

# load sas data

In [ ]:
from sas7bdat import SAS7BDAT
with SAS7BDAT(oj(data_dir, 'DATA', 'ahrf2019.sas7bdat'), skip_header=False) as reader:
    df = reader.to_data_frame()
df.to_pickle(oj(data_dir, 'processed', 'df.pkl'))

# save metadata

In [11]:
import AHRF_parser #;reload(AHRF_parser)
ascii_file_path = oj(data_dir, 'DATA/AHRF2019.asc')
sas_file_path = oj(data_dir, "DOC/ahrf2018-19.sas")

In [ ]:
# Takes as input number of cores to use
ahrf_parser = AHRF_parser.parse_AHRF_ascii(num_cores=1,
                                           ascii_file_path=ascii_file_path,
                                           sas_file_path=sas_file_path)

# will generate meta_data.csv - need to move this to the right location

# rename features

In [26]:
df = pd.read_pickle(oj(data_dir, 'processed', 'df.pkl'))
meta = pd.read_csv(oj(data_dir, 'processed', 'meta_data.csv'))
meta_dict = {}
for i in range(meta.shape[0]):
    r = meta.iloc[i]
    meta_dict[r.FieldId] = r.FieldName
df = df.rename(columns=meta_dict)
df.to_pickle(oj(data_dir, 'processed', 'df_renamed.pkl'))

# analyze data

In [28]:
df = pd.read_pickle(oj(data_dir, 'processed', 'df_renamed.pkl'))

In [29]:
df.head()

,Blank,Header-FIPSStandCtyCode,EntityofFile,SecondaryEntityOfFile,DateofFile,DateofCreation,FileLength,StateName,StateNameAbbreviation,CountyName,...,Daysw/8-hrAvgOzoneovrNAAQSMaximum8-hourAverageOzone2014,Daysw/8-hrAvgOzoneovrNAAQSMaximum8-hourAverageOzone2013,Daysw/8-hrAvgOzoneovrNAAQSMaximum8-hourAverageOzone2012,Daysw/8-hrAvgOzoneovrNAAQSMaximum8-hourAverageOzone2011,Daysw/8-hrAvgOzoneovrNAAQSMaximum8-hourAverageOzone2010,Daysw/8-hrAvgOzoneovrNAAQSMaximum8-hourAverageOzone2009,Daysw/8-hrAvgOzoneovrNAAQSMaximum8-hourAverageOzone2008,Daysw/8-hrAvgOzoneovrNAAQSMaximum8-hourAverageOzone2007,Daysw/8-hrAvgOzoneovrNAAQSMaximum8-hourAverageOzone2006,ElevationFeet1976
0,,01001,AHRF,01001,2019,19212.0,31661.0,Alabama,AL,Autauga,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,5.0,3.0,290.0
1,,01003,AHRF,01003,2019,19212.0,31661.0,Alabama,AL,Baldwin,...,2.0,1.0,0.0,2.0,2.0,1.0,2.0,7.0,14.0,155.0
2,,01005,AHRF,01005,2019,19212.0,31661.0,Alabama,AL,Barbour,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,3.0,220.0
3,,01007,AHRF,01007,2019,19212.0,31661.0,Alabama,AL,Bibb,...,0.0,0.0,0.0,0.0,2.0,0.0,2.0,12.0,7.0,224.0
4,,01009,AHRF,01009,2019,19212.0,31661.0,Alabama,AL,Blount,...,0.0,0.0,2.0,1.0,1.0,0.0,4.0,10.0,7.0,870.0
